# Minimum Cost Diet Problem: Group Casimir Funk

## Topic:

## Objectives:

## Table of Contents:
* [[A] Description of population of interest](#description)
* [[A] Dietary Reference Intakes ](#dri)
* [[A] Data on prices for different foods](#prices)
* [[A] Nutritional content of different foods](#nutrition)
* [[A] Solution](#solution)

#### Setup

In [109]:
!pip install -r requirements.txt --upgrade

import pandas as pd
from eep153_tools.sheets import read_sheets
import fooddatacentral as fdc
import warnings
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

In [110]:
# API key for Gov; substitute your own!
apikey = "Hfo8BxP2zgYjatcMv4zuDmmCkzhiwzWWNJYE3iOQ"

## [A] Description of population of interest <a class="anchor" id="description"></a>

## [A] Dietary Reference Intakes  <a class="anchor" id="dri"></a>

Write a function that takes as arguments the characteristics of a
person (e.g., age, sex) and returns a pandas.Series of Dietary
Reference Intakes (DRI's) or Recommended Daily Allowances
(RDA) of a variety of nutrients appropriate for your population of
interest.

## [A] Data on prices for different foods <a class="anchor" id="prices"></a>

Retrieves our google spreadsheet of different prices for both raw meat and raw vegan diets, and concatenates them into a Dataframe.

In [111]:
sheets = [("https://docs.google.com/spreadsheets/d/1GuSUsrGc5GX6hPerOCEJTDN22wVadD62NtyRLToH-Ho/edit#gid=0",
           "Ingredients")]

df = read_sheets(sheets[0][0])[sheets[0][1]]

Key available for students@eep153.iam.gserviceaccount.com.


### Raw Meat Diet prices

In [112]:
raw_meat_df = df.loc[df["RAW MEAT"] == "v"]
raw_meat_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
0,Beef,1.00,lb,12.99,2-22-2023,Wholefood,746760,,v
1,Pork,1.00,lb,9.99,2-26-2023,Wholefood,749420,,v
2,"Chicken, drumstick",1.00,lb,3.99,2-26-2023,Wholefood,331897,,v
3,Canned Tuna,5.00,oz,0.99,2-27-2023,Wholefood,334194,,v
4,Atlantic Salmon,1.00,lb,15.99,2-27-2023,Wholefood,175167,,v
5,Bison,16.00,oz,10.99,2-27-2023,Wholefood,175293,,v
6,Bacon,1.00,lb,7.99,2-26-2023,Wholefood,749420,,v
7,Eggs,270.00,oz,5.99,2-26-2023,Wholefood,748967,,v
8,Turkey,1.00,lb,12.99,2-26-2023,Wholefood,746785,,v
9,Sausage,1.00,lb,6.99,2-26-2023,Wholefood,746783,,v


### Raw Vegan Diet prices

In [113]:
raw_vegan_df = df.loc[df["RAW VEGAN"] == "v"]
raw_vegan_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
18,Russet Potato,1.0,lb,1.69,2-22-2023,https://www.wholefoodsmarket.com/product/produ...,170027,v,v
19,Pasta,16.0,oz,1.59,2-22-2023,https://www.wholefoodsmarket.com/product/365-b...,172014,v,v
20,"Lemon, raw",58.0,grams,0.99,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2344662,v,v
21,"Lime, raw",67.0,grams,0.69,2023-2-27,https://www.wholefoodsmarket.com/product/produ...,2344664,v,v
22,"Nuts, pecans, halves, raw",7.0,oz,11.79,2023-2-27,https://www.wholefoodsmarket.com/product/auror...,2346395,v,v
...,...,...,...,...,...,...,...,...,...
78,"Mangos, raw",336.0,grams,1.09,2023-2-27,Safeway,169910,v,
79,Chia seeds,8.0,oz,7.99,2023-2-27,https://www.wholefoodsmarket.com/product/navit...,2343065,v,
80,Flax seeds,16.0,oz,7.89,2023-2-27,safeway,2343063,v,
81,"Quinoa, fat added",16.0,oz,3.79,2023-2-27,https://www.safeway.com/shop/product-details.9...,2343885,v,


### USDA Diet prices

In [114]:
usda_df = df[(df["RAW MEAT"] == "v") | (df["RAW VEGAN"] == "v")]
usda_df

,Food,Quantity,Units,Price,Date,Location,FDC,RAW VEGAN,RAW MEAT
0,Beef,1.0,lb,12.99,2-22-2023,Wholefood,746760,,v
1,Pork,1.0,lb,9.99,2-26-2023,Wholefood,749420,,v
2,"Chicken, drumstick",1.0,lb,3.99,2-26-2023,Wholefood,331897,,v
3,Canned Tuna,5.0,oz,0.99,2-27-2023,Wholefood,334194,,v
4,Atlantic Salmon,1.0,lb,15.99,2-27-2023,Wholefood,175167,,v
...,...,...,...,...,...,...,...,...,...
78,"Mangos, raw",336.0,grams,1.09,2023-2-27,Safeway,169910,v,
79,Chia seeds,8.0,oz,7.99,2023-2-27,https://www.wholefoodsmarket.com/product/navit...,2343065,v,
80,Flax seeds,16.0,oz,7.89,2023-2-27,safeway,2343063,v,
81,"Quinoa, fat added",16.0,oz,3.79,2023-2-27,https://www.safeway.com/shop/product-details.9...,2343885,v,


## [A] Nutritional content of different foods <a class="anchor" id="nutrition"></a>

Now we have a list of foods with prices.  Do lookups on USDA database
to get nutritional information.

In [115]:
## This function utilizes the FDC API to do nutritional content lookups for every food in our Data Frame.
def getNutritionalContent(diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        diet_df = raw_meat_df
    elif diet == "vegan":
        diet_df = raw_vegan_df
    elif diet == "usda":
        diet_df = usda_df
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    D = {}
    for food in  diet_df.Food.tolist():
        try:
            FDC = diet_df.loc[diet_df.Food==food,:].FDC.iloc[0]
            D[food] = fdc.nutrients(apikey,FDC).Quantity
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s or server is overwhelmed. Try again later!" % (food,FDC))        

    return pd.DataFrame(D,dtype=float)

### Raw Meat Diet Nutritional content

In [116]:
Raw_Meat_FoodNutrients = getNutritionalContent("meat")
Raw_Meat_FoodNutrients

,Beef,Pork,"Chicken, drumstick",Canned Tuna,Atlantic Salmon,Bison,Bacon,Eggs,Turkey,Sausage,...,Russet Potato,Pasta,"Lemon, raw","Lime, raw","Nuts, pecans, halves, raw","Nuts, walnuts, English, halves, raw","Cashews, NFS","Pistachio nuts, NFS","Almond butter, creamy","Peanut butter, creamy"
25-hydroxycholecalciferol,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.560,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alanine,NaN,NaN,NaN,NaN,1.271,1.348,NaN,0.667,NaN,0.94,...,NaN,0.195,NaN,NaN,0.4375,0.59,NaN,NaN,1.181,1.160
"Alcohol, ethyl",NaN,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
Amino acids,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000,NaN,0.00,...,0.0,0.000,NaN,NaN,0.0000,0.00,NaN,NaN,0.000,0.000
Arginine,NaN,NaN,NaN,NaN,1.221,1.377,NaN,0.787,NaN,1.03,...,NaN,0.211,NaN,NaN,1.3630,2.02,NaN,NaN,2.771,3.323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Raw Vegan Diet Nutritional content

In [117]:
Raw_Vegan_FoodNutrients = getNutritionalContent("vegan")
Raw_Vegan_FoodNutrients

,Russet Potato,Pasta,"Lemon, raw","Lime, raw","Nuts, pecans, halves, raw","Nuts, walnuts, English, halves, raw","Cashews, NFS","Pistachio nuts, NFS","Almond butter, creamy","Peanut butter, creamy",...,"Apples, honeycrisp, with skin, raw","Oranges, raw, navels","Strawberries, raw","Blueberries, raw","Raspberries, raw","Mangos, raw",Chia seeds,Flax seeds,"Quinoa, fat added","WHITE SLICED MUSHROOMS, WHITE"
Alanine,NaN,0.195,NaN,NaN,0.4375,0.590,NaN,NaN,1.181,1.160,...,NaN,0.028,NaN,NaN,NaN,0.082,NaN,NaN,NaN,NaN
"Alcohol, ethyl",NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.0,0.0,NaN
Amino acids,0.00,0.000,NaN,NaN,0.0000,0.000,NaN,NaN,0.000,0.000,...,NaN,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN
Arginine,NaN,0.211,NaN,NaN,1.3630,2.020,NaN,NaN,2.771,3.323,...,NaN,0.099,NaN,NaN,NaN,0.031,NaN,NaN,NaN,NaN
Ash,1.13,0.550,NaN,NaN,1.4390,1.641,NaN,NaN,3.185,2.771,...,0.0865,0.430,0.345,0.2288,0.35,0.360,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### USDA Diet Nutritional content

In [118]:
USDA_FoodNutrients = getNutritionalContent("usda")
USDA_FoodNutrients

,Beef,Pork,"Chicken, drumstick",Canned Tuna,Atlantic Salmon,Bison,Bacon,Eggs,Turkey,Sausage,...,"Apples, honeycrisp, with skin, raw","Oranges, raw, navels","Strawberries, raw","Blueberries, raw","Raspberries, raw","Mangos, raw",Chia seeds,Flax seeds,"Quinoa, fat added","WHITE SLICED MUSHROOMS, WHITE"
25-hydroxycholecalciferol,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.560,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alanine,NaN,NaN,NaN,NaN,1.271,1.348,NaN,0.667,NaN,0.94,...,NaN,0.028,NaN,NaN,NaN,0.082,NaN,NaN,NaN,NaN
"Alcohol, ethyl",NaN,NaN,NaN,NaN,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.0,0.0,NaN
Amino acids,NaN,NaN,NaN,NaN,0.000,0.000,NaN,0.000,NaN,0.00,...,NaN,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN
Arginine,NaN,NaN,NaN,NaN,1.221,1.377,NaN,0.787,NaN,1.03,...,NaN,0.099,NaN,NaN,NaN,0.031,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cis-Lutein/Zeaxanthin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cis-beta-Carotene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trans-Lycopene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## [A] Solution <a class="anchor" id="solution"></a>

What is the minimum cost diet for our population? How much does it cost, and of what does it consist? How
does it vary with age, sex, and level of activity?

### Units & Prices

In [119]:
### We use the `units` function to convert all foods to either deciliters or hectograms, to match FDC database
def getUnitsAndPrices(diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        diet_df = raw_meat_df
    elif diet == "vegan":
        diet_df = raw_vegan_df
    elif diet == "usda":
        diet_df = usda_df
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    # Convert food quantities to FDC units
    diet_df['FDC Quantity'] = diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    diet_df['FDC Price'] = diet_df['Price']/diet_df['FDC Quantity']

    diet_df.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    diet_prices = diet_df.groupby('Food',sort=False)['FDC Price'].min()

    return diet_prices

### Raw Meat Diet Units & Prices

In [120]:
meat_prices = getUnitsAndPrices("meat")
meat_prices

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/tmp/ipykernel_47/263509696.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Quantity'] = diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
/tmp/ipykernel_47/263509696.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Price'] = diet_df['Price']/diet_df['FDC Quant

Food
Beef                                    2.8638047857815594 / hectogram
Pork                                    2.2024179992269266 / hectogram
Chicken, drumstick                      0.8796444261176615 / hectogram
Canned Tuna                              0.698424446601692 / hectogram
Atlantic Salmon                          3.525191572336192 / hectogram
Bison                                   2.4228802614118043 / hectogram
Bacon                                   1.7614934748571718 / hectogram
Eggs                                   0.07825593780666174 / hectogram
Turkey                                  2.8638047857815594 / hectogram
Sausage                                  1.541031212672294 / hectogram
Butter                                   2.641137900974833 / hectogram
Raw Liver                                0.659182163932784 / hectogram
Whole milk                             0.16627189797876257 / hectogram
Yogurt                                  0.5500533441512695 / hectogram
2

### Raw Vegan Diet Units & Prices

In [121]:
vegan_prices = getUnitsAndPrices("vegan")
vegan_prices

/tmp/ipykernel_47/263509696.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Quantity'] = diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
/tmp/ipykernel_47/263509696.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Price'] = diet_df['Price']/diet_df['FDC Quantity']


Food
Russet Potato                    0.37258122309244307 / hectogram
Pasta                            0.35053499687395534 / hectogram
Lemon, raw                        1.7068965517241381 / hectogram
Lime, raw                         1.0298507462686566 / hectogram
Nuts, pecans, halves, raw          5.941143019793614 / hectogram
                                              ...               
Mangos, raw                       0.3244047619047619 / hectogram
Chia seeds                        3.5229869497143436 / hectogram
Flax seeds                        1.7394472486386838 / hectogram
Quinoa, fat added                 0.8355519736806859 / hectogram
WHITE SLICED MUSHROOMS, WHITE      1.318364327865568 / hectogram
Name: FDC Price, Length: 63, dtype: object

### USDA Diet Units & Prices

In [122]:
usda_prices = getUnitsAndPrices("usda")
usda_prices

/tmp/ipykernel_47/263509696.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Quantity'] = diet_df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))
/tmp/ipykernel_47/263509696.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diet_df['FDC Price'] = diet_df['Price']/diet_df['FDC Quantity']


Food
Beef                             2.8638047857815594 / hectogram
Pork                             2.2024179992269266 / hectogram
Chicken, drumstick               0.8796444261176615 / hectogram
Canned Tuna                       0.698424446601692 / hectogram
Atlantic Salmon                   3.525191572336192 / hectogram
                                              ...              
Mangos, raw                      0.3244047619047619 / hectogram
Chia seeds                       3.5229869497143436 / hectogram
Flax seeds                       1.7394472486386838 / hectogram
Quinoa, fat added                0.8355519736806859 / hectogram
WHITE SLICED MUSHROOMS, WHITE     1.318364327865568 / hectogram
Name: FDC Price, Length: 81, dtype: object

### Dietary Requirements

Our constraints come from US government recommendations, which is available to view at https://www.dietaryguidelines.gov/sites/default/files/2021-03/Dietary_Guidelines_for_Americans-2020-2025.pdf

In [123]:
DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


### Using `solve_subsistence_problem` to analyze diet

In [124]:
def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index,axis=0)
    idx = Amin.index.to_frame()
    idx['type'] = 'min'
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [125]:
## This prints out the solution to the minimum diet cost problem for a given population and diet.
def getPopulationSolution(group, diet):
    
    diet = diet.lower()
    
    if diet == "meat":
        nutrients_df = Raw_Meat_FoodNutrients
        diet_prices = meat_prices
    elif diet == "vegan":
        nutrients_df = Raw_Vegan_FoodNutrients
        diet_prices = vegan_prices
    elif diet == "usda":
        nutrients_df = USDA_FoodNutrients
        diet_prices = usda_prices
    else:
        print("Diet not found, please choose from meat, vegan, or usda.")
        return
    
    tol = 1e-6

    result = solve_subsistence_problem(nutrients_df,diet_prices,diet_min[group],diet_max[group],tol=tol)

    print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

    # Put back into nice series
    diet = result.diet

    print("\nDiet (in 100s of grams or milliliters):")
    print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
    print()

    tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
    print("\nWith the following nutritional outcomes of interest:")
    print(tab)
    print()

    print("\nConstraining nutrients are:")
    excess = tab.diff(axis=1).iloc[:,1]
    print(excess.loc[np.abs(excess) < tol*100].index.tolist())

### Results

#### Raw Meat Diet Results

In [129]:
getPopulationSolution("M 19-30", "meat")
getPopulationSolution("F 19-30", "meat")

Cost of diet for M 19-30 is $16.83 per day.


Diet (in 100s of grams or milliliters):
Raw Liver                0.430678
2% milk                  2.531126
Lime, raw                2.942569
Cashews, NFS             2.842878
Pistachio nuts, NFS      1.356587
Almond butter, creamy    1.168707
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          3100.000000          2400.0
Protein                          118.169883            56.0
Fiber, total dietary              42.098166            33.6
Folate, DFE                      400.000000           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      201.677519           130.0
Iron, Fe                          31.726653             8.0
Magnesium, Mg                   1257.499889           400.0
Niacin                            18.729

#### Raw Vegan Diet Results

In [130]:
getPopulationSolution("M 19-30", "vegan")
getPopulationSolution("F 19-30", "vegan")

Cost of diet for M 19-30 is $0.00 per day.


Diet (in 100s of grams or milliliters):
Series([], dtype: float64)


With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                              NaN          2400.0
Protein                             NaN            56.0
Fiber, total dietary                NaN            33.6
Folate, DFE                         NaN           400.0
Calcium, Ca                         NaN          1000.0
Carbohydrate, by difference         NaN           130.0
Iron, Fe                            NaN             8.0
Magnesium, Mg                       NaN           400.0
Niacin                              NaN            16.0
Phosphorus, P                       NaN           700.0
Potassium, K                        NaN          4700.0
Riboflavin                          NaN             1.3
Thiamin                             NaN          

/tmp/ipykernel_47/3925801159.py:74: UserWarning: The problem is (trivially) infeasible due to a row of zeros in the equality constraint matrix with a nonzero corresponding  constraint value.
  warnings.warn(result.message)
/tmp/ipykernel_47/3925801159.py:74: UserWarning: The problem is (trivially) infeasible due to a row of zeros in the equality constraint matrix with a nonzero corresponding  constraint value.
  warnings.warn(result.message)


##### USDA Diet Results

In [131]:
getPopulationSolution("M 19-30", "usda")
getPopulationSolution("F 19-30", "usda")

Cost of diet for M 19-30 is $4.91 per day.


Diet (in 100s of grams or milliliters):
Eggs                     1.020657
Raw Liver                0.167902
2% milk                  6.140118
Peanut butter, creamy    0.691030
Collards, raw            0.018342
Avocado, raw             5.270175
Black beans, NFS         0.000434
Lentils, NFS             0.389884
Oranges, raw, navels     0.616611
Flax seeds               0.152580
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          3100.000000          2400.0
Protein                           77.153199            56.0
Fiber, total dietary              52.964177            33.6
Folate, DFE                      485.843411           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      130.000000           130.0
Iron, Fe                 